# Model Building and Training

In this notebook we will train our model on mixed data through multiple inputs.

In [1]:
# conda install h5py==2.10.0

In [24]:
sorted([2,1,3,4,5])

[1, 2, 3, 4, 5]

In [2]:
import tarfile
import platform
import pickle
import pandas as pd
import bz2
from datetime import datetime,timedelta
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import concatenate
import cv2
import os
import glob
from data_processing_functions import *

%load_ext autoreload
%autoreload 2
%aimport data_processing_functions

In [2]:
# if platform.system() == "Darwin" and platform.processor() == "arm":
#     # Force enable GPU in ARM Macs for now
#     from tensorflow.python.compiler.mlcompute import mlcompute
#     mlcompute.set_mlc_device(device_name="gpu")

### Read in Data:

In [4]:
model_data_dict = open_pickle('../../data/model_data_dict.pkl')

In next 3 cells open the images from the tar.bz2 files using the `load_sky_images()` function. This will be a numpy array of all of the images for each instance in the df. We aren't using the built in stream from directory function of keras due to the file type. The files are very large, so I would prefer not to unzip them on my machine

In [5]:
img_5_min = model_data_dict['df_5_min']['file']

In [6]:
sorted_img_5 = img_5_min.sort_values()

In [7]:
sorted_img_5

18        2014/01/02/20140102_154607.jpg
31        2014/01/02/20140102_155908.jpg
221       2014/01/02/20140102_190915.jpg
274       2014/01/02/20140102_200215.jpg
321       2014/01/02/20140102_204915.jpg
                       ...              
764599    2016/12/30/20161230_204100.jpg
764686    2016/12/30/20161230_220759.jpg
764701    2016/12/30/20161230_222259.jpg
764712    2016/12/30/20161230_223400.jpg
764843    2016/12/31/20161231_004500.jpg
Name: file, Length: 10000, dtype: object

In [12]:
# sorted_img_5.drop(['level_0','index'],axis=1,inplace=True)

In [77]:
images_5_min = load_sky_images(sorted_img_5)

2014 data/Folsom_sky_images_2014.tar.bz2
deleted tar
2015 data/Folsom_sky_images_2015.tar.bz2
deleted tar
2016 data/Folsom_sky_images_2016.tar.bz2


In [78]:
save_pickle('../data_rp/5_min_bigger_images',images_5_min)

In [3]:
# save_pickle('../data_rp/5_min_w_img.pkl',images_5_min)
# images_5_min.shape
images_5_min = open_pickle('../../data/5_min_bigger_images')

In [8]:
train_ind = model_data_dict['df_5_min']['X_train'].index
test_ind = model_data_dict['df_5_min']['X_test'].index

In [24]:
train_ind

Int64Index([156459, 213193, 148457, 764843, 568289, 621850, 258805,  80907,
            486393,  76508,
            ...
            603483, 656081, 429103, 394452, 707767, 589827,  48099, 454651,
            570033,   9537],
           dtype='int64', length=8000)

In [9]:
# train_ind.to_list().index(18)
train_ind.shape

(8000,)

In [39]:
test_ind.to_list().index(18)
test_ind.shape

(2000,)

Figure out how to get the tensors sorted in the right order now:

In [11]:
# test_checker = {}
trainImagesX = np.zeros([8000,64,64], dtype = int)
testImagesX = np.zeros([2000,64,64], dtype = int)

train_ind = train_ind.to_list()
test_ind = test_ind.to_list()
i = 0
for ind in sorted_img_5.index:
    try:
        img_ind = train_ind.index(ind)
#         test_checker[img_ind] = i
        trainImagesX[img_ind] = images_5_min[i]
        i += 1
    except:
        img_ind = test_ind.index(ind)
#         test_checker[img_ind] = i
        testImagesX[img_ind] = images_5_min[i]
        i += 1
        

In [62]:
# sorted_img_5 = pd.DataFrame(images_5_min).reset_index()
# sorted_img_5['img_matrix'] = images_5_min

In [25]:
# sorted_img_5 images_5_min.shape

In [22]:
# train_img_ind,test_img_ind = get_index_of_img(train_ind,test_ind,sorted_img_5)
# trainImagesX,testImagesX = get_img_from_index(train_img_ind,test_img_ind,images_5_min)

### Process our image data and targets:

right now, we'll just start by scaling the images down from [0-255], to between [0-1].

In [12]:
# images_5_min = images_5_min / 255.0
trainImagesX = trainImagesX / 255.0
testImagesX = testImagesX / 255.0

In [74]:
# model_data_dict['df_5_min'].keys()

In [13]:
# find the largest irradiation in the training set and use it to
# scale our irradiation to the range [0, 1] (will lead to better
# training and convergence)
maxInt = model_data_dict['df_5_min']['y_train'].max()
trainY = model_data_dict['df_5_min']['y_train'] / maxInt
testY = model_data_dict['df_5_min']['y_test'] / maxInt

save processed data:

In [67]:
save_pickle('../../data/training_ImagesX.pkl',trainImagesX)
save_pickle('../../data/testing_ImagesX.pkl',testImagesX)
save_pickle('data/TrainY.pkl',trainY)
save_pickle('data/testY.pkl',testY)

### Build model:

The three parts of this section will be making adjustments to pre-trained CNN models, building an MLP, and then adding layers at the end to contatinate the results of these two branches of NN models, and then added fully connected and regression layers at the end.

So, let's break them out: 
1. [Update pre-trained CNN models for image data](#Update-pretrained-CNN-models)
2. [Build MLP for numeric/categorical data](#Build-MLP-for-numeric-and-categorical-data)
3. [Create end of multi-imput model](#Build-the-final-section-of-our-model)

Opening data:

In [14]:
trainImagesX = open_pickle('../../data/training_ImagesX.pkl')
testImagesX = open_pickle('../../data/testing_ImagesX.pkl')
trainY = open_pickle('data/TrainY.pkl',)
testY = open_pickle('data/testY.pkl',)
testY = open_pickle('data/testY.pkl')

#### Update pretrained CNN models

Starting with ResNet50, but will do the others next.

In [15]:
# import ssl

# ssl._create_default_https_context = ssl._create_unverified_context

baseModelResNet = ResNet50(weights="imagenet", include_top=False,
                           input_tensor=Input(shape=((64, 64, 3))))
# trainImagesX.shape[0],
# input_shape=(trainImagesX.shape[0],32, 32, 3)

# baseModelIv3 = InceptionV3(weights="imagenet", include_top=False,
#                             input_tensor=Input(shape=(1,32, 32, 3)))

# baseModelResIv2 = InceptionResNetV2(weights="imagenet", include_top=False,
#                                       input_tensor=Input(shape=(32, 32, 3)))

In [16]:
baseModelResNet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [17]:
chanDim = -1

# flatten the volume, then FC => RELU => BN => DROPOUT
headModel = baseModelResNet.output
# headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten()(headModel)
headModel = Dense(16)(headModel)
headModel = Activation("relu")(headModel)
headModel = BatchNormalization(axis=chanDim)(headModel)
headModel = Dropout(0.5)(headModel)
# apply another FC layer, this one to match the number of nodes
# coming out of the MLP
headModel = Dense(4)(headModel)
headModel = Activation("relu")(headModel)


baseModelResNet.trainable = False

cnn = Model(inputs=baseModelResNet.input, outputs=headModel)

#### Lets test with just a CNN:

In [18]:
chanDim = -1

# flatten the volume, then FC => RELU => BN => DROPOUT
headModel2 = baseModelResNet.output
# headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel2 = Flatten()(headModel2)
headModel2 = Dense(16)(headModel2)
headModel2 = Activation("relu")(headModel2)
headModel2 = BatchNormalization(axis=chanDim)(headModel2)
headModel2 = Dropout(0.5)(headModel2)
# apply another FC layer, this one to match the number of nodes
# coming out of the MLP
headModel2 = Dense(4)(headModel)
headModel2 = Activation("relu")(headModel2)
headModel2 = Dense(1, activation="linear")(headModel2)


baseModelResNet.trainable = False

cnn2 = Model(inputs=baseModelResNet.input, outputs=headModel)

8000

In [19]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
cnn2.compile(loss="mean_absolute_percentage_error", optimizer=opt)

cnn2.fit(
    x=trainImagesX, y=trainY,
    validation_data=(testImagesX, testY),
    epochs=200, batch_size=8)

Epoch 1/200


ValueError: in user code:

    /Users/elenamorais/miniforge3/envs/tensorflow_m1_env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/elenamorais/miniforge3/envs/tensorflow_m1_env/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/elenamorais/miniforge3/envs/tensorflow_m1_env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/elenamorais/miniforge3/envs/tensorflow_m1_env/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/elenamorais/miniforge3/envs/tensorflow_m1_env/lib/python3.8/site-packages/tensorflow/python/distribute

#### Build MLP for numeric and categorical data

We're going to start with something super basic initially to see how this does.

In [11]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(4, activation="relu"))
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
        # return our model
        
    return model

In [27]:
trainAttrX = model_data_dict['df_5_min']['X_train_p']
mlp = create_mlp(trainAttrX.shape[1], regress=False)

#### Build the final section of our model

Fully connected and with linear activation function for regression output.

In [28]:
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [29]:
testAttrX = model_data_dict['df_5_min']['X_test_p']

In [30]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(
    x=[trainAttrX, trainImagesX], y=trainY,
    validation_data=([testAttrX, testImagesX], testY),
    epochs=200, batch_size=8)

[INFO] training model...
Epoch 1/200


ValueError: in user code:

    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/elena.morais/Documents/environments/reseach_paper/lib/python3.7/site-packages/tensorfl

In [ ]:
# make predictions on the testing data
print("[INFO] predicting irridiation...")
preds = model.predict([testAttrX, testImagesX])